In [19]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

# Data handling and preprocessing
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Image processing
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Deep Learning
import keras
from keras import layers, models
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Data utilities
import pickle
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

In [13]:
import sys
print(sys.executable)

/Users/apple/Documents/ML/trafficSigns/.venv/bin/python
